In [1]:
from pyfuseki import FusekiQuery, FusekiUpdate
from pysolr import Solr
import httpx
from rdflib import Graph
from api.src.routes.translate.makeTranslate import MakeTranslate
from api.src.function.loc.graphExist import GraphExist
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from api.src.function.authorities.makeGraph import MakeGraphSubject

In [ ]:
fuseki_query = FusekiQuery('http://localhost:3030', 'authorities')

In [33]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">599</int>\n</lst>\n</response>\n'

In [2]:
authority = 'http://id.loc.gov/authorities/subjects/sh85124200'

In [5]:
url = f'http://localhost:8000/authorities/loc/subject?uri={authority}'
r = httpx.get(url)
r.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-05-13',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-13T10:17:41',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh85124200'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Sociologia', 'lang': 'pt'}}],
 'note': 'Aqui são inseridos trabalhos sobre a sociologia como ramo do saber. Esta rubrica pode ser dividida geograficamente para trabalhos neste ramo de conhecimento em um local específico. Trabalhos sobre as condições sociais de determinadas regiões, países, cidades, etc., são inseridos sob o nome do local subdividido por [Condições sociais.

In [30]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201

In [11]:
graph = Graph()
graph.parse(f'{authority}.rdf')
# graph.serialize('Sociology.ttl')

<Graph identifier=N0a185217859a43b69269d035ef7e65a8 (<class 'rdflib.graph.Graph'>)>

In [3]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

def GetUriBK(url):
    token = url.split("/")[-1]
    queryBK = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
      PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
      SELECT ?uri ?label WHERE {{ graph ?g {{
          ?uri madsrdf:authoritativeLabel ?label .
        ?sub bf:identifiedBy ?identifiedBy .
          ?identifiedBy rdf:value "{token}"
          FILTER EXISTS {{ ?identifiedBy bf:assigner <http://id.loc.gov/vocabulary/organizations/dlc> }}
      }} }} """
    response = fuseki_query.run_sparql(queryBK)
    [binding] = response.convert()['results']['bindings']
    uri = {"value": binding['uri']['value'],
              "label": {
                "value": binding['label']['value'],
                "lang": "pt"
              } }
    return uri

def GetUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          url = i.get('value')
          if metadado in ['hasBroaderAuthority', 'hasNarrowerAuthority']:
            exist = GraphExist(url)
            if exist:
              uri = GetUriBK(url)
            else:
              uri = {
                    "value": url,
                    "label": {
                  "value": i.get('label').value,
                  "lang": i.get('label').language
                } }
          else:
            uri = {
                  "value": url,
                   "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj



In [ ]:
request = Subject(**r.json())
request

In [10]:
for variant in request.hasVariant:
    print(variant)

type='Topic' elementList=[Element(type='TopicElement', elementValue=Label(value='Social theory', lang='en'))] componentList=None


In [ ]:
graph = MakeGraphSubject(request, "TESTE")
print(graph)

In [2]:
fuseki_query = FusekiQuery('http://localhost:3030', 'authorities')
token = 'sh85066150'
query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT ?uri ?label WHERE {{ graph ?g {{
            ?sub bf:identifiedBy ?identifiedBy .
    ?identifiedBy rdf:value "{token}" .
    ?uri madsrdf:authoritativeLabel ?label .   
                }} }} """
responseQuery = fuseki_query.run_sparql(query)
exist = responseQuery.convert()['results']['bindings'][0]

In [4]:
exist = GraphExist("http://id.loc.gov/authorities/subjects/sh85066150")
exist

{'uri': {'type': 'uri',
  'value': 'https://bibliokeia.com/authorities/Topic/bkau-2'},
 'label': {'type': 'literal', 'value': 'Ciência da informação'}}

In [56]:
# Type 
def GetType(graph, authority):

  qtype = f"""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
              PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
              PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
              SELECT ?type WHERE {{ 
                  <{authority}> rdf:type ?type .
                  FILTER ( ?type != madsrdf:Authority ) 
                  FILTER ( ?type != skos:Concept )
                  }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]
  return tipo

# hasBroaderAuthority
def GetHasBroader(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasBroaderAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
               print(exist)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(external) > 0:
            obj['hasBroaderExternalAuthority'] = external
    return obj

# hasNarrowerAuthority
def GetHasNarrower(authority, graph, obj):

    ask = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasNarrowerAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(ask)
    if len(response.bindings) > 0:
        external = list()
        internal = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
                uri = {
                   "value": exist['uri']['value'],
                   "base": "bk",
                   "label": {
                       "value": exist['label']['value'],
                       "lang": "pt" } }
                internal.append(uri)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(internal) > 0:
            obj['hasNarrowerAuthority'] = internal
        if len(external) > 0:
            obj['hasNarrowerExternalAuthority'] = external
    return obj

# hasNarrowerExternalAuthority
def GetHasNarrowerExternal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasNarrowerExternalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value')
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        return external
    else:
        return False

# hasCloseExternalAuthority
def GetCloseExternal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasCloseExternalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           base = url.split("//")[1].split("/")[0]
           uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
           external.append(uri)
        obj['hasCloseExternalAuthority'] = external

# hasExactExternalAuthority
def GetExactExternal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasExactExternalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           base = url.split("//")[1].split("/")[0]
           uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
           external.append(uri)
        obj['hasExactExternalAuthority'] = external
    return obj

In [39]:
def GetHasReciprocal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasReciprocalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
               print(exist)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(external) > 0:
            obj['hasReciprocalExternalAuthority'] = external
    return obj

In [15]:
def GetVariant(graph, obj):
  qVariant = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
                <{authority}> madsrdf:hasVariant ?variant .
                ?variant rdf:type ?typeVariant .
                ?variant madsrdf:elementList ?elementList .
                ?elementList rdf:rest* ?node .
                  ?node rdf:first ?e .
                  ?e madsrdf:elementValue ?elementValue .
                ?e rdf:type ?typeElement .
                FILTER ( ?typeVariant != madsrdf:Variant )
                }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants
  return obj
obj = GetVariant(graph, {})
obj


{'hasVariant': [{'type': 'Topic',
   'elementList': [{'type': 'TopicElement',
     'elementValue': {'value': 'Social theory', 'lang': 'en'}}]}]}

In [14]:
for i in r.bindings:
    print(i)

{rdflib.term.Variable('typeVariant'): rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#Topic'), rdflib.term.Variable('elementValue'): rdflib.term.Literal('Social theory', lang='en'), rdflib.term.Variable('typeElement'): rdflib.term.URIRef('http://www.loc.gov/mads/rdf/v1#TopicElement')}


In [ ]:
def ParserSubject(graph, authority):
      
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  tipo = GetType(graph, authority)
  

  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]      
  }
  # ElementList
  qElementList = f"""{prefix}
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
  r = graph.query(qElementList)
  elementList = list()
  for i in r.bindings:
      element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('elementValue').value,
            "lang": i.get('elementValue').language
          }
        }
      elementList.append(element)

  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "elementList": elementList,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }

  # Note 
  qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
  r = graph.query(qNote)
  if len(r.bindings) > 0:
     obj['note'] = r.bindings[0].get('note').value

# hasVariant
  qVariant = f"""{prefix}
  SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
	<{authority}> madsrdf:hasVariant ?variant .
  ?variant rdf:type ?typeVariant .
  ?variant madsrdf:elementList ?elementList .
  ?elementList rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
	?e rdf:type ?typeElement .
  FILTER ( ?typeVariant != madsrdf:Variant )
  }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants

  # URIS
  # hasBroaderAuthority
  obj = GetHasBroader(authority, graph, obj)
  # Narrower Terms
  obj = GetHasNarrower(authority, graph, obj)
  hasNarrower = GetHasNarrowerExternal(authority, graph, obj)
  if hasNarrower:
    obj['hasNarrowerExternalAuthority'] = obj['hasNarrowerExternalAuthority']+hasNarrower
  # ExactExternal
  obj = GetExactExternal(authority, graph, obj)
  # CloseExternal
  obj = GetCloseExternal(authority, graph, obj)

  # hasReciprocalAuthority
  response = Subject(**obj)

  return response
request = ParserSubject(graph, authority)
request

In [ ]:
request.hasNarrowerExternalAuthority

In [56]:

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201